In [1]:
import os, sys
from pathlib import Path
import cortex
import nibabel as nib
import numpy as np
import matplotlib.colors as colors
import matplotlib.pyplot as pl
import time
import platform
import subprocess

In [2]:
if platform.node()=='theCryingMachine.local':
    data_dir = Path('/Users/ningkai/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/')
elif platform.node()=='Smoldering-Corpse-Bar.local':
    data_dir = Path('/Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/')
elif platform.node()=='node230':
    raise ValueError('hmmm I dont think it works on a server. Your are working on {}'.format(platform.node()))
else:
    raise ValueError('No idea which computer this is. The computer name is {}'.format(platform.node()))

In [3]:
subj_nr = 2
sub = 'sub-{0:03d}'.format(subj_nr)
cx_sub = "EO_" + sub

fs_dir = data_dir / 'freesurfer'
fs_subj_dir = Path(fs_dir, sub)

vonmises_dir = data_dir / 'vonmises'
example_func_fn = data_dir / 'func' / 'sub-002_ses-01_task-ping_run-4_space-T1w_boldref.nii.gz'
bmask_fn = data_dir / 'func' / 'sub-002_ses-01_task-ping_run-4_space-T1w_desc-brain_mask.nii.gz'
bmask = nib.load(bmask_fn)

bref = nib.load(example_func_fn)

ctx_dir = data_dir / 'cortex' / 'outputs'

In [4]:
print('cortex.database.default_filestore: {}'.format(cortex.database.default_filestore))
print('cortex.options.usercfg: {}'.format(cortex.options.usercfg))

cortex.database.default_filestore: /Users/dionysus/Documents/pycortex/db
cortex.options.usercfg: /Users/dionysus/Library/Application Support/pycortex/options.cfg


In [5]:
cortex.freesurfer.import_subj(fs_subject=sub,
                                cx_subject=cx_sub,
                                freesurfer_subject_dir=fs_dir)

ValueError: Do not overwrite

In [6]:
flirt_mtx = np.loadtxt(data_dir / "register/flirt.mtx",)
print(flirt_mtx)
bbregdat = [[-1.000000000000000e+00, 2.055505901807919e-04, 6.701818347210065e-05, 1.680365443229675e+00],
[6.702871178276837e-05, 4.870421616942622e-05, 1.000000119209290e+00, -1.140921497344971e+01], 
[-2.055470977211371e-04, -1.000000119209290e+00, 4.871539204032160e-05, -8.093381881713867e+00],
[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00, 1.000000000000000e+00]]

[[ 1.00000000e+00 -2.05550000e-04 -6.70300000e-05  4.57238388e+01]
 [-6.70200000e-05  4.87200000e-05 -9.99999880e-01  1.84187180e+02]
 [ 2.05550000e-04  9.99999880e-01  4.87000000e-05  2.49198628e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  9.99999880e-01]]


In [7]:
custom_affine = np.identity(4)
# custom_affine[1,3] -= 8
transform = cortex.xfm.Transform(custom_affine, bref)
transform.save(cx_sub, 'fmriprep', 'magnet')

In [8]:
T1 = Path(fs_dir, sub, "mri", "T1.nii.gz")
T2 = Path(fs_dir, sub, "mri", "T2.nii.gz")
T12 = Path(fs_dir, sub, "mri", "T12.nii.gz")
for f, fn in zip([T1, T2], ['T1', 'T2']):
    if f.exists():
        print('{} exists'.format(fn))
    else:
        os.system("mri_convert %s %s" % (Path(str(fn).replace('.nii.gz', '.mgz')), f))
        print('convert {}'.format(fn))

if not T12.exists():
    os.system("fslmaths %s -div %s %s" % (T1, T2, T12))
    print('create T12')
else:
    print('T12 exists')

aref = nib.load(T1)
transform = cortex.xfm.Transform(np.identity(4), aref)
transform.save(cx_sub, 'anat', 'magnet')

T1 exists
T2 exists
T12 exists


In [9]:
f'export SUBJECTS_DIR={fs_dir}'

'export SUBJECTS_DIR=/Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/suppr_pRF/data/spinoza/sub-002/derivatives/freesurfer'

In [10]:
os.environ["SUBJECTS_DIR"] = f'{fs_dir}'
a = cortex.xfm.Transform.from_fsl(flirt_mtx, example_func_fn, T1)

In [11]:
a.xfm

array([[ 5.92417047e-01, -1.21773272e-04, -3.97037916e-05,
         4.50617018e+01],
       [ 1.21769405e-04,  5.92417119e-01, -2.88461959e-05,
         5.98900943e+01],
       [ 3.94294219e-05,  2.86381307e-05,  5.88235344e-01,
         3.43375463e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000012e+00]])

In [12]:
R2_percentile = 97

prf_rsq = nib.load(vonmises_dir / 'prf_rsq.nii.gz')
prf_rsq_data = prf_rsq.get_fdata()

retinamap = nib.load(vonmises_dir / 'prf_best_angle.nii.gz')
prf_rsq_data[~np.array(bmask.get_fdata(), dtype=bool)] = np.nan
q = np.nanpercentile(prf_rsq_data, R2_percentile)
prf_rsq_data[prf_rsq_data<q] = np.nan
retinamap_data = retinamap.get_fdata()
retinamap_data[np.isnan(prf_rsq_data)] = np.nan 
retinamap_thred = nib.Nifti1Image(retinamap_data, retinamap.affine, retinamap.header)
R2_data_thred = nib.Nifti1Image(prf_rsq_data, prf_rsq.affine, prf_rsq.header)

In [13]:
np.nanmax(prf_rsq_data)

0.401932954788208

In [14]:
## make some test data
# left hemisphere is all 100 and right hemisphere is all 0
test_data = prf_rsq_data.copy()

test_data[0:int(test_data.shape[0]/2),:,:] = 100
test_data[int(test_data.shape[0]/2):,:,:] = 0
test_data[~np.array(bmask.get_fdata(), dtype=bool)] = np.nan

In [15]:
rsq_file = vonmises_dir / 'sub-002_ses-1_space-fsnative_hemi-LR_desc-prf_rsq.npy'

rsq_data = np.load(rsq_file)
rsq_vcx = cortex.Vertex(rsq_data, subject=cx_sub, vmin=0, vmax=0.3, cmap="magma")

In [16]:
retinamap_file = vonmises_dir / 'sub-002_ses-1_space-fsnative_hemi-LR_desc-prf_best_angle.npy'
retinamap_data = np.load(retinamap_file)
retinamap_data[rsq_data<0.03] = np.nan
retinamap_vcx = cortex.Vertex(retinamap_data, subject=cx_sub, vmin=0, vmax=360, cmap="hsv")

In [17]:
kappa_file = vonmises_dir / 'sub-002_ses-1_space-fsnative_hemi-LR_desc-prf_best_kappa.npy'
kappa_data = np.load(kappa_file)
kappa_data[rsq_data<0.03] = np.nan
kappa_vcx = cortex.Vertex(kappa_data, subject=cx_sub, vmin=0, vmax=50, cmap="J4s")

In [18]:
cortex.webgl.show({"retinamap": retinamap_vcx, "rsq": rsq_vcx, "kappa": kappa_vcx})

Started server on port 22251
Stopping server


<JS: window.viewer>

In [19]:
T12_anat_vcx = cortex.Volume(str(fs_subj_dir / 'mri' / 'T12.nii.gz'), cx_sub, 'anat', cmap='Greens', vmin=0, vmax=1)
prf_best_angle_vcx = cortex.Volume(data=retinamap_data.transpose((2,1,0)), subject=cx_sub, xfmname='fmriprep', cmap='hsv', vmin=0, vmax=360)
prf_best_kappa_vcx = cortex.Volume(str(vonmises_dir / 'prf_best_kappa.nii.gz'), cx_sub, 'fmriprep', cmap='Greens', vmin=0, vmax=50)
t1w_volume_vcx = cortex.Volume(str(fs_subj_dir / 'mri' / 'T1.nii.gz'), cx_sub, 'anat')
prf_rsq_vcx = cortex.Volume(data=prf_rsq_data.transpose((2,1,0)), subject=cx_sub, xfmname='fmriprep', cmap='hot', vmin=0, vmax=0.3)
ds = {"prf_best_angle": prf_best_angle_vcx, 
      "myelin": T12_anat_vcx, 
      "prf_best_kappa": prf_best_kappa_vcx, 
      "t1w": t1w_volume_vcx,
      "prf_rsq": prf_rsq_vcx}
# for cope, name in zip((1,), ["prf_rsq"]):
#     cope_fn = vonmises_dir / (name + '.nii.gz')
#     cope_vcx = cortex.Volume(str(cope_fn), cx_sub, 'fmriprep', cmap='hot')
#     ds[name] = cope_vcx

ds['testdata'] = cortex.Volume(data=test_data.transpose((2,1,0)), subject=cx_sub, xfmname='fmriprep', cmap='spring', vmin=0, vmax=100)

cortex.webgl.show(ds)
cortex.webgl.make_static(ctx_dir / sub, ds)

ValueError: axes don't match array